<a href="https://colab.research.google.com/github/VeraLi0710/USS-study/blob/main/W05_quiz_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 5 Quiz

This notebook contains the SQL Quiz for Week 5. Section 1 uses the New York City data we know and love. Section 2 will conduct a spatial damage assessment across the Gaza strip.

### INSTRUCTIONS:

Run this notebook in Google Colab. The answer to each quesiton will be a number or a string. Input these into the corresponding question on Moodle. You have 90 minutes to attempt the quiz, so if you get stuck on a question, move on.

Make sure you run all of the cells of code in order. If you run into serious problems, try clicking on the "runtime" tab above and selecting "restart session and run all".

# Section 1



This URL: `https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip` points to a .zip file containing shapefiles associated with the NYC data that we've been using in class so far. You can choose how to use sql, either the `con.sql()` syntax or the `%%sql` syntax. Either way, use the code cell below to install and import the necessary packages.

In [1]:
%pip install duckdb duckdb-engine jupysql leafmap

In [2]:
import duckdb
import leafmap

In [3]:
# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [4]:
%%sql
duckdb:///:memory:
INSTALL spatial;
LOAD spatial;

,Success


## Question 1

Firslty, please provide the link to your colab notebook in the Moodle quiz using the following steps:

1. In the top right corner, click the "share" button
2. In the popup, click on "restricted" and change this to "anyone with the link"
3. Finally, click "copy link", and paste the link into the box for Question 1 on moodle.

Once you've done this, download and unzip the data, then create the following tables using the corresponding shapefiles.

- nyc_neighborhoods
- nyc_census_blocks
- nyc_homicides
- nyc_streets
- nyc_subway_stations

In [5]:
url='https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip'
leafmap.download_file(url, unzip=True)

Downloading...
From: https://s3.amazonaws.com/s3.cleverelephant.ca/postgis-workshop-2020.zip
To: /content/postgis-workshop-2020.zip
100%|██████████| 22.5M/22.5M [00:01<00:00, 18.8MB/s]


Extracting files...


'/content/postgis-workshop-2020.zip'

In [8]:
%%sql

CREATE TABLE nyc_neighborhoods AS SELECT * FROM "postgis-workshop/data/nyc_neighborhoods.shp";
CREATE TABLE nyc_census_blocks AS SELECT * FROM "postgis-workshop/data/nyc_census_blocks.shp";
CREATE TABLE nyc_homicides AS SELECT * FROM "postgis-workshop/data/nyc_homicides.shp";
CREATE TABLE nyc_streets AS SELECT * FROM "postgis-workshop/data/nyc_streets.shp";
CREATE TABLE nyc_subway_stations AS SELECT * FROM "postgis-workshop/data/nyc_subway_stations.shp";

,Success


## Question 2:
What is the longest street in New York? Note, it may be split up into multiple segments! Ignore missing and null values.


In [10]:
%%sql

SELECT * FROM nyc_streets;

,ID,NAME,ONEWAY,TYPE,geom
0,1,Shore Pky S,None,residential,"[1, 4, 0, 0, 0, 0, 0, 0, 23, 66, 15, 73, 202, ..."
1,2,None,None,footway,"[1, 4, 0, 0, 0, 0, 0, 0, 80, 57, 15, 73, 35, 1..."
2,3,Avenue O,None,residential,"[1, 4, 0, 0, 0, 0, 0, 0, 228, 63, 15, 73, 219,..."
3,4,Walsh Ct,None,residential,"[1, 4, 0, 0, 0, 0, 0, 0, 139, 62, 15, 73, 102,..."
4,5,None,None,motorway_link,"[1, 4, 0, 0, 0, 0, 0, 0, 176, 53, 15, 73, 16, ..."
...,...,...,...,...,...
19086,17378,FDR Dr,yes,motorway,"[1, 4, 0, 0, 0, 0, 0, 0, 191, 114, 14, 73, 206..."
19087,17382,FDR Dr,yes,motorway,"[1, 4, 0, 0, 0, 0, 0, 0, 18, 115, 14, 73, 192,..."
19088,19039,FDR Dr,None,motorway,"[1, 4, 0, 0, 0, 0, 0, 0, 98, 50, 15, 73, 233, ..."
19089,19053,FDR Dr,None,motorway,"[1, 4, 0, 0, 0, 0, 0, 0, 59, 51, 15, 73, 94, 1..."


In [14]:
%%sql

SELECT NAME, total_length
FROM (
    SELECT NAME, SUM(ST_Length(geom)) AS total_length
    FROM nyc_streets
    WHERE geom IS NOT NULL AND NAME IS NOT NULL
    GROUP BY NAME
) AS subquery
ORDER BY total_length DESC
LIMIT 1;

,NAME,total_length
0,Grand Central Pky,40210.484038


## Question 3:

Which borough had the fewest daytime shootings in 2009?

In [15]:
%%sql

SELECT * FROM nyc_homicides

,INCIDENT_D,BORONAME,NUM_VICTIM,PRIMARY_MO,ID,WEAPON,LIGHT_DARK,YEAR,geom
0,2008-01-01,Brooklyn,1,None,7,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2008-01-04,Manhattan,1,None,14,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,2008-01-05,Queens,1,None,15,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,2008-01-04,Queens,1,None,16,knife,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,2008-01-05,Queens,1,None,18,gun,D,2008,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...
3979,2010-10-11,The Bronx,1,None,4269,gun,None,2010,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3980,2010-10-06,The Bronx,1,None,4271,knife,None,2010,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3981,2011-07-26,The Bronx,1,None,4282,gun,None,2011,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3982,2011-07-28,The Bronx,1,None,4284,gun,None,2011,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [17]:
%%sql

SELECT BORONAME, COUNT(*) AS shooting_count
FROM nyc_homicides
WHERE YEAR = 2009 AND WEAPON = 'gun' AND INCIDENT_D BETWEEN '2009-01-01 06:00:00' AND '2009-12-31 18:00:00'
GROUP BY BORONAME
ORDER BY shooting_count ASC
LIMIT 1;

,BORONAME,shooting_count
0,Staten Island,9


## Question 4:

What is the total population of the census blocks served by the L train?

In [19]:
#Display the table
%%sql

SELECT * FROM nyc_census_blocks

,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geom
0,360850009001000,97,51,32,1,5,8,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151, 8..."
1,360850020011000,66,52,2,0,7,5,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72, ..."
2,360850040001000,62,14,18,2,25,3,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 82, 227, 12, 73, 55, ..."
3,360850074001000,137,92,12,0,13,20,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 204, 85, 13, 73, 103,..."
4,360850096011000,289,230,0,0,32,27,Staten Island,"[2, 4, 0, 0, 0, 0, 0, 0, 107, 247, 12, 73, 7, ..."
...,...,...,...,...,...,...,...,...,...
38789,360050295001004,328,267,14,2,8,37,The Bronx,"[2, 4, 0, 0, 0, 0, 0, 0, 118, 130, 16, 73, 15,..."
38790,360050295002002,0,0,0,0,0,0,The Bronx,"[2, 4, 0, 0, 0, 0, 0, 0, 244, 138, 16, 73, 135..."
38791,360050419004001,0,0,0,0,0,0,The Bronx,"[2, 4, 0, 0, 0, 0, 0, 0, 78, 3, 17, 73, 85, 28..."
38792,360050255002001,480,96,96,20,12,256,The Bronx,"[2, 4, 0, 0, 0, 0, 0, 0, 4, 123, 16, 73, 154, ..."


In [20]:
#Display the table
%%sql

SELECT * FROM nyc_subway_stations

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geom
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,487.0,909.0,JFK Terminal 8,None,None,"JFK Terminal 8, Queens",JFK Terminal 8,Queens,None,None,None,AIR-BLUE,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
487,488.0,903.0,Federal Circle,Rental Car,None,"Federal Circle / Rental Car, Queens",Federal Circle / Rental Car,Queens,None,None,None,AIR-BLUE,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
488,489.0,902.0,Long Term Parking,None,None,"Long Term Parking, Queens",Long Term Parking,Queens,None,None,None,AIR-BLUE,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
489,490.0,901.0,Howard Beach,None,159th Ave,"Howard Beach, Queens",Howard Beach,Queens,None,None,A,AIR-BLUE,None,None,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [27]:
%%sql

SELECT SUM(p.POPN_TOTAL) AS total_population
FROM nyc_census_blocks p
JOIN nyc_subway_stations s
ON ST_Intersects(p.geom, s.geom)
WHERE s.ROUTES LIKE '%L%';

,total_population
0,7147.0


## Question 5

which subway station had the largest number of homicides within 1km?

In [28]:
#Display the table
%%sql

SELECT * FROM nyc_streets

,ID,NAME,ONEWAY,TYPE,geom
0,1,Shore Pky S,None,residential,"[1, 4, 0, 0, 0, 0, 0, 0, 23, 66, 15, 73, 202, ..."
1,2,None,None,footway,"[1, 4, 0, 0, 0, 0, 0, 0, 80, 57, 15, 73, 35, 1..."
2,3,Avenue O,None,residential,"[1, 4, 0, 0, 0, 0, 0, 0, 228, 63, 15, 73, 219,..."
3,4,Walsh Ct,None,residential,"[1, 4, 0, 0, 0, 0, 0, 0, 139, 62, 15, 73, 102,..."
4,5,None,None,motorway_link,"[1, 4, 0, 0, 0, 0, 0, 0, 176, 53, 15, 73, 16, ..."
...,...,...,...,...,...
19086,17378,FDR Dr,yes,motorway,"[1, 4, 0, 0, 0, 0, 0, 0, 191, 114, 14, 73, 206..."
19087,17382,FDR Dr,yes,motorway,"[1, 4, 0, 0, 0, 0, 0, 0, 18, 115, 14, 73, 192,..."
19088,19039,FDR Dr,None,motorway,"[1, 4, 0, 0, 0, 0, 0, 0, 98, 50, 15, 73, 233, ..."
19089,19053,FDR Dr,None,motorway,"[1, 4, 0, 0, 0, 0, 0, 0, 59, 51, 15, 73, 94, 1..."


In [29]:
%%sql

SELECT s.name AS station_name, COUNT(h.id) AS homicide_count
FROM nyc_subway_stations s
LEFT JOIN nyc_homicides h
ON ST_Distance(s.geom, h.geom) <= 1000
GROUP BY s.name
ORDER BY homicide_count DESC
LIMIT 1;

,station_name,homicide_count
0,125th St,305


# Section 2

This section explores building damage in the Gaza Strip resulting from the ongoing war. You will conduct a geospatial analysis of building damage using two datasets:

1. [Humantiarian Open Street Map](https://www.hotosm.org/projects/gaza-building-footprints-pre-conflict-update-2024/) building footprints for the Gaza Strip
  * data: https://storage.googleapis.com/qm2/Gaza_Buildings_2.geojson.zip
2. [Damage points](https://unosat.org/products/3984) from the UN Satellite Agency (UNOSAT).
  * data: https://storage.googleapis.com/qm2/UNOSAT_GAZA_20240503_2.zip


The Coordinate Reference System of these datasets is EPSG:4326; therefore, stock functions like ST_DISTANCE() will yield values in degrees, not meters. Be mindful of this in your analysis, and use the DuckDB spatial functions [documentation](https://duckdb.org/docs/extensions/spatial/functions#st_geomfromtext) to your advantage.


First, download and unzip the data, and create two tables: `gaza_buildings`, containing the building footprint data, and `damage_points` containing the UNOSAT damage points. Make sure to set a spatial index on these tables, this will make your queries run much faster!

In [5]:
url='https://storage.googleapis.com/qm2/Gaza_Buildings_2.geojson.zip'
leafmap.download_file(url, unzip=True)

Gaza_Buildings_2.geojson.zip already exists. Skip downloading. Set overwrite=True to overwrite.


'/content/Gaza_Buildings_2.geojson.zip'

In [6]:
url='https://storage.googleapis.com/qm2/UNOSAT_GAZA_20240503_2.zip'
leafmap.download_file(url, unzip=True)

UNOSAT_GAZA_20240503_2.zip already exists. Skip downloading. Set overwrite=True to overwrite.


'/content/UNOSAT_GAZA_20240503_2.zip'

In [7]:
%%sql

CREATE TABLE gaza_buildings AS
SELECT *
FROM ST_Read('Gaza_Buildings_2.geojson');

CREATE INDEX gaza_buildings_geometry_idx
ON gaza_buildings
USING RTREE(geom);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [8]:
%%sql

CREATE TABLE damage_points AS
SELECT *
FROM ST_Read('/content/UNOSAT_GAZA_20240503_2.shp');


CREATE INDEX damage_points_geometry_idx
ON damage_points
USING RTREE(geom);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [9]:
%%sql

SELECT * FROM gaza_buildings;

,osm_id,osm_type,building,name,geom
0,1279368365,ways_poly,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 5, 113, 9, 66, 94, 4,..."
1,1279368369,ways_poly,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 248, 112, 9, 66, 39, ..."
2,1279368371,ways_poly,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 77, 113, 9, 66, 48, 6..."
3,1279378354,ways_poly,greenhouse,None,"[2, 4, 0, 0, 0, 0, 0, 0, 125, 108, 9, 66, 115,..."
4,1279378372,ways_poly,greenhouse,None,"[2, 4, 0, 0, 0, 0, 0, 0, 18, 115, 9, 66, 34, 2..."
...,...,...,...,...,...
335179,17758232,relations,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 131, 44, 9, 66, 108, ..."
335180,3997601,relations,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 183, 153, 9, 66, 5, 1..."
335181,17489427,relations,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 155, 154, 9, 66, 14, ..."
335182,17614218,relations,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 142, 82, 9, 66, 16, 1..."


In [10]:
%%sql

SELECT * FROM damage_points LIMIT 5;

,OBJECTID,SiteID,SensorDate,SensorID,Confidence,Main_Damag,SensorDa_1,SensorID_2,Confiden_1,Main_Dam_1,...,Damage_S_5,Grouped_Da,FieldValid,Notes,Territory,Governorat,Municipali,Neighborho,EventCode,geom
0,1.0,26,None,NaN,NaN,NaN,2023/11/07 00:00:00.000,8,1,1,...,0,1,0,None,Gaza Strip,North Gaza,Jabalya,Az Zohour,CE20231007PSE,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2.0,26,None,NaN,NaN,NaN,2023/11/07 00:00:00.000,8,1,1,...,0,1,0,None,Gaza Strip,North Gaza,Jabalya,Az Zohour,CE20231007PSE,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,3.0,26,None,NaN,NaN,NaN,2023/11/07 00:00:00.000,8,2,3,...,0,1,0,None,Gaza Strip,North Gaza,Jabalya,Az Zohour,CE20231007PSE,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,4.0,26,None,NaN,NaN,NaN,2023/11/07 00:00:00.000,8,2,3,...,0,1,0,None,Gaza Strip,North Gaza,Jabalya,Az Zohour,CE20231007PSE,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,5.0,26,None,NaN,NaN,NaN,2023/11/07 00:00:00.000,8,2,3,...,0,1,0,None,Gaza Strip,North Gaza,Jabalya,Az Zohour,CE20231007PSE,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


## Question 6

Create a table called "gaza_buildings_damaged" which is comprised only of building footprints that intersect with a UNOSAT damage point. How many damaged buildings are there in Gaza?

In [11]:
%%sql

CREATE TABLE gaza_buildings_damaged AS
SELECT b.*
FROM gaza_buildings AS b
JOIN damage_points AS d
ON ST_Intersects(b.geom, d.geom);

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [12]:
%%sql

SELECT * FROM  gaza_buildings_damaged;

,osm_id,osm_type,building,name,geom
0,295795550,ways_poly,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 56, 225, 8, 66, 100, ..."
1,295795519,ways_poly,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 127, 225, 8, 66, 81, ..."
2,1289276701,ways_poly,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 146, 225, 8, 66, 12, ..."
3,295795533,ways_poly,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 148, 225, 8, 66, 185,..."
4,295806622,ways_poly,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 195, 234, 8, 66, 234,..."
...,...,...,...,...,...
109490,295971223,ways_poly,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 230, 48, 10, 66, 33, ..."
109491,1301937739,ways_poly,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 1, 49, 10, 66, 244, 5..."
109492,1301746412,ways_poly,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 233, 48, 10, 66, 71, ..."
109493,296106799,ways_poly,yes,None,"[2, 4, 0, 0, 0, 0, 0, 0, 231, 48, 10, 66, 147,..."


In [13]:
%%sql

-- Caculate the damage buildings
SELECT COUNT(*) AS damaged_building_count
FROM gaza_buildings_damaged;

,damaged_building_count
0,109495


## Question 7

What is the total area (in square kilometers) of damaged buildings in Gaza? note: there are TWO area functions in duckDB, and only one of them returns answers in meters.

In [14]:
%%sql

SELECT SUM(ST_Area(geom)) / 1000000 AS total_area_sq_km
FROM gaza_buildings_damaged;

,total_area_sq_km
0,2.213315e-09


## Question 8

What percentage of hospitals in Gaza have been damaged?


In [15]:
%%sql
-- Caculate the total of hospital
WITH total_hospitals AS (
    SELECT COUNT(*) AS total_count
    FROM gaza_buildings
    WHERE building = 'hospital'
),

-- Caculate the count of damaged hospital
damaged_hospitals AS (
    SELECT COUNT(*) AS damaged_count
    FROM gaza_buildings_damaged
    WHERE building = 'hospital'
)

-- Caculate the percentage of hospitals in Gaza have been damaged
SELECT
    (dh.damaged_count * 100.0 / th.total_count) AS percentage_damaged_hospitals
FROM
    total_hospitals th,
    damaged_hospitals dh;

,percentage_damaged_hospitals
0,65.384615


## Question 9

The UNOSAT data contains a column called "Governorat" (shapefiles only allow column names of up to 10 characters). Calculate the number of damaged schools in each governorate.

In [16]:
%%sql

SELECT dp.Governorat, COUNT(*) AS damaged_school_count
FROM gaza_buildings_damaged AS d
JOIN damage_points AS dp
ON ST_Intersects(d.geom, dp.geom)
WHERE d.building = 'school'
GROUP BY dp.Governorat
ORDER BY damaged_school_count DESC;

,Governorat,damaged_school_count
0,Gaza,97
1,Khan Yunis,50
2,North Gaza,35
3,Deir Al-Balah,6


## Question 10

The "Dar Al-Shifa Hospital" was the largest medical complex in the gaza strip. Find the hospital's building footprint, and calculate the proportion of damaged buildings within 1 kilometer.

In [17]:
%%sql

SELECT geom
FROM gaza_buildings
WHERE name = 'Dar Al-Shifa Hospital';

,geom
0,"[2, 4, 0, 0, 0, 0, 0, 0, 190, 197, 9, 66, 124,..."


In [36]:
%%sql

WITH hospital AS (
    SELECT geom
    FROM gaza_buildings
    WHERE name = 'Dar Al-Shifa Hospital'
),

total_buildings AS (
    SELECT COUNT(*) AS total_count
    FROM gaza_buildings
    WHERE ST_DWithin(geom, (SELECT geom FROM hospital), 1000)
),

damaged_buildings AS (
    SELECT COUNT(*) AS damaged_count
    FROM gaza_buildings_damaged
    WHERE ST_DWithin(geom, (SELECT geom FROM hospital), 1000)
)


SELECT
    (db.damaged_count * 100.0 / tb.total_count) AS proportion_damaged
FROM
    total_buildings tb,
    damaged_buildings db;

,proportion_damaged
0,32.667132
